In [1]:
import os
from github import Github
import re
import sqlparse
import json
import time

In [2]:
def extract_ctes(sql_code):
    # Regular expression to capture CTE name and the SQL following AS
    cte_regex = re.compile(r"(\w+)\s+AS\s+\(", re.IGNORECASE)
    
    # Find all matches for CTEs
    matches = cte_regex.finditer(sql_code)
    
    ctes = []
    
    # Iterate through each match to extract the CTE
    for match in matches:
        cte_name = match.group(1)
        start_idx = match.end()  # Starting index after 'AS ('
        end_idx = None
        parentheses_count = 1  # We already have one opening parenthesis
        
        in_single_quote = False
        in_double_quote = False
        escape_char = False
        
        # Iterate through the remaining SQL to find matching parentheses
        for idx in range(start_idx, len(sql_code)):
            char = sql_code[idx]
            
            # Handle escape character inside a string
            if escape_char:
                escape_char = False  # Skip the next character after a backslash
                continue
            
            # Check for the escape character
            if char == '\\':
                escape_char = True
                continue
            
            # Toggle flags for entering/exiting single or double quotes
            if char == "'" and not in_double_quote:
                in_single_quote = not in_single_quote
                continue
            elif char == '"' and not in_single_quote:
                in_double_quote = not in_double_quote
                continue
            
            # If we're inside a string, ignore any parentheses
            if in_single_quote or in_double_quote:
                continue
            
            # Track parentheses when not inside a string
            if char == '(':
                parentheses_count += 1
            elif char == ')':
                parentheses_count -= 1
            
            # If parentheses are balanced, we've found the end of the CTE definition
            if parentheses_count == 0:
                end_idx = idx
                break
        
        # Extract the CTE SQL code, excluding the last parenthesis
        if end_idx is not None:
            cte_sql = sql_code[start_idx:end_idx].strip()  # Exclude the last closing parenthesis
            ctes.append({
                'name': cte_name,
                'sql': cte_sql
            })
    
    return ctes

In [8]:
# Authenticate with GitHub
token = os.getenv('GITHUB_TOKEN')
g = Github(token)

def search_github_for_ctes(query, language="SQL", sort="indexed", order="desc", limit=1000):
    # Search GitHub for code matching the query
    results = g.search_code(query=query, language=language, sort=sort, order=order)
    
    ctes_list = []

    try:
        print("searching for CTEs:")
        print("[",end='')
        for file in results[:limit]:
            try:
                # Check rate limit
                rate_limit = g.get_rate_limit().search
                if rate_limit.remaining == 0:
                    print("Rate limit exceeded. Returning results gathered so far.")
                    break

                # Get the contents of the file
                content = file.decoded_content.decode('utf-8')

                
                repo_full_name = file.repository.full_name
                file_repo = file.repository
                repo = g.get_repo(repo_full_name)
                
                # Parse the SQL content using sqlparse
                parsed = sqlparse.parse(content)
                
                for statement in parsed:
                    # Find CTEs using regex
                    matches = extract_ctes(str(statement))
                    
                    for match in matches:
                        cte_name = match['name'].strip()
                        cte_query_body = match['sql'].strip()
                        # ctes_list.append({cte_name: cte_query_body})

                        ctes_list.append(
                            {
                                'cte_name': cte_name,
                                'SQL': cte_query_body,
                                'Repo': repo_full_name,
                                'file_name': file.name,
                                'file_path':file.html_url,
                                'file_size':file.size,
                                'repo_path':file_repo.html_url,
                                'repo_description': repo.description,
                                'repo_size': repo.size,
                                'num_commits': repo.get_commits().totalCount,
                                'num_branches': repo.get_branches().totalCount,
                                'num_prs': repo.get_pulls(state='all').totalCount,
                                'repo_contributors': repo.get_contributors().totalCount,
                                'repo_releases': repo.get_releases().totalCount,
                                'repo_stars': file_repo.stargazers_count,
                                'repo_forks': file_repo.forks_count,
                            }
                        )

            except Exception as e:
                print(f"Error processing file {file.html_url}: {e}")
            print("*",end='')
        print("]")

    except Exception as e:
        print(f"Exception occurred: {e}")
        
    return ctes_list


schemas = [
    "Chinook",
    "Northwind",
    "AdventureWorks",
    "Sakila",
    "HR",
    "Pagila",
    "DVD Rental",
    "World",
    "Employees",
    "TPC-H",
    "IMDB",
    "FoodMart",
    "WideWorldImporters",
    "ClassicModels",
    "Airline"
]

# prepare informationfor retrieval
query_json = [{'query': f'language:SQL WITH {schema}', 'schema': {schema}} for schema in schemas]


In [27]:
list_to_test = search_github_for_ctes('WITH IMDB', limit=10)

searching for CTEs:
[**********]


In [ ]:
list_to_test

In [4]:
# Convert lists and sets within the dictionaries to tuples to ensure hashability
def make_hashable(d):
    return {k: tuple(v) if isinstance(v, (list, set)) else v for k, v in d.items()}

In [5]:
# Function to check if schema data is already in the file
def schema_exists_in_file(schema, file_name='combined_data.json'):
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            data = json.load(file)
            return any(item.get('schema') == schema for item in data)
    return False

# Function to save the combined data to the JSON file
def save_to_json(data, file_name='combined_data.json'):
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            existing_data = json.load(file)
    else:
        existing_data = []
    
    # Combine new and existing data, removing duplicates
    combined_data = existing_data + data
    unique_data = [dict(t) for t in {tuple(sorted(make_hashable(d).items())) for d in combined_data}]

    # Save the unique data to the JSON file
    with open(file_name, 'w') as file:
        json.dump(unique_data, file, indent=4)

In [6]:
def generate_github_dataset(name_of_file: str):
    combined = []
    mod_2_flag = 0

    for query_schema_object in query_json:
        schema = query_schema_object["schema"]

        # Check if the schema data is already in the file
        if schema_exists_in_file(schema, name_of_file):
            print(f'Schema {schema} already processed, skipping...')
            continue

        print(f'\nGetting data for {schema} schema')

        try:
            ctes_desc = search_github_for_ctes(query_schema_object['query'])
            ctes_asc = search_github_for_ctes(query_schema_object['query'], order='asc')
            ctes = ctes_desc + ctes_asc

            # Add the schema attribute
            for obj in ctes:
                obj['schema'] = schema

            # Combine results and remove duplicates
            combined += ctes
            unique_list = [dict(t) for t in {tuple(sorted(make_hashable(d).items())) for d in combined}]

            # Save progress to JSON file
            save_to_json(unique_list, name_of_file)

            mod_2_flag += 1

        except Exception as e:
            print(f'Error occurred for schema {schema}: {e}')
            break  # Handle the error and break to avoid losing progress

In [ ]:
# Remove duplicates by converting the list to a set of tuples
unique_list = [dict(t) for t in {tuple(sorted(make_hashable(d).items())) for d in combined}]

# Saving the CTEs to a JSON file
with open(name_of_file, 'w') as json_file:
    json.dump(unique_list, json_file, indent=4)

In [9]:
file_name = 'github/github_ctes.json'
generate_github_dataset(file_name)


Getting data for {'Chinook'} schema
searching for CTEs:
[******************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

Request GET /repos/fvgm-spec/learn_dbt/pulls?state=all&per_page=1 failed with 403: Forbidden
Setting next backoff to 881.021136s


*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************]

Getting data for {'N

Request GET /repos/jasonwnc/ds2002F23/pulls?state=all&per_page=1 failed with 403: Forbidden
Setting next backoff to 785.586909s


*************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

...

**********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************]
searching for CTEs:
[*******************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

Request GET /repositories/727882504/contents/blog-akasa.sql?ref=dcfce379db0a97603a5e4c6afa98e1c703ab4e21 failed with 403: Forbidden
Setting next backoff to 33.500035s


******************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************]
searching for CTEs:
[***********************************************

Request GET /repos/Yona-p/sql_cs50 failed with 403: Forbidden
Setting next backoff to 290.092441s


***************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************]
